In [20]:
excel_file_path = 'z:/Q-Figures/T32-Poster/Nate/T7-Summary(LocalOld).xlsx'

In [ ]:
import pandas as pd
excel_file_path = 'z:/Q-Figures/T32-Poster/Nate/T7-Summary(LocalOld).xlsx'
# Replace 'your_file.xlsx' with the actual path to your Excel file
# Read the Excel file
data = pd.read_excel(excel_file_path, skiprows=[0])

# Define the indices for the columns you're interested in (0-based indexing)
column2_index = 1  # Replace with the correct index for column 2
column3_index = 2  # Replace with the correct index for column 3

# Filter out rows where column 3 has a value but column 2 is blank
filtered_data = data[data.iloc[:, column2_index].notna() | data.iloc[:, column3_index].isna()]

# Extract the second and third columns from the filtered data and convert them to lists
second_column = filtered_data.iloc[:, column2_index].tolist()
third_column = filtered_data.iloc[:, column3_index].tolist()

# Remove 'NaN' values from the lists
indices = [int(value) for value in second_column if not pd.isna(value)]
case_numbers = [int(value) for value in third_column if not pd.isna(value)]

print("Index:", indices)
print("Case:", case_numbers)


In [ ]:
import os

root_directory = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/"

# Iterate through each case number
for case_number in case_numbers:
    case_folder = os.path.join(root_directory, f"Case-{case_number:02d}-")
    
    # Check if the case folder exists
    if os.path.exists(case_folder):
        # Search for the required files in the case folder
        vx3_file = os.path.join(case_folder, "Vx3.nrrd")
        label_file = [file for file in os.listdir(case_folder) if file.endswith("label.nrrd")]

        # Check if both files exist
        if os.path.exists(vx3_file) and label_file:
            print(f"Files found for Case-{case_number:02d}-: Vx3.nrrd and {label_file[0]}")
        else:
            print(f"Files not found for Case-{case_number:02d}-")
    else:
        print(f"Case folder not found for Case-{case_number:02d}-")



In [ ]:
import os

# Iterate through each case number
for case_number in case_numbers:
    case_folder_prefix = f"Case-{case_number:02d}-"
    
    # Iterate through subfolders in the root directory
    for folder_name in os.listdir(root_directory):
        if folder_name.startswith(case_folder_prefix):
            case_folder = os.path.join(root_directory, folder_name)
            
            # Search for the required files in the case folder
            vx3_file = os.path.join(case_folder, "Vx3.nrrd")
            label_file = [file for file in os.listdir(case_folder) if file.endswith("label.nrrd")]

            # Check if both files exist
            if os.path.exists(vx3_file) and label_file:
                print(f"Files found for {folder_name}: Vx3.nrrd and {label_file[0]}")
            else:
                print(f"Files not found for {folder_name}")


In [ ]:
import os


# Function to search for files recursively within a folder
def search_files(folder_path):
    vx3_file = os.path.join(folder_path, "Vx3.nrrd")
    label_files = [file for file in os.listdir(folder_path) if file.endswith("label.nrrd")]

    return vx3_file, label_files

# Iterate through each case number
for case_number in case_numbers:
    case_folder_prefix = f"Case-{case_number:02d}-"
    
    # Iterate through subfolders in the root directory
    for folder_name in os.listdir(root_directory):
        if folder_name.startswith(case_folder_prefix):
            case_folder = os.path.join(root_directory, folder_name)

            # Search for files within subfolders
            for root, dirs, files in os.walk(case_folder):
                vx3_file, label_files = search_files(root)

                # Check if both files exist
                if os.path.exists(vx3_file) and label_files:
                    print(f"Files found for {root}: Vx3.nrrd and {label_files[0]}")
                
# perform analysist

In [ ]:
#1 FINAL WORKING CODE FOR IMAGE GENERATION FOR ONE CASE


#############################################################################################################################################################
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import nrrd
from PIL import Image, ImageDraw
import os
import vtk
from mayavi import mlab

# QC IMAGE STUFF
# Casename = "H-14479_JH110088_SPI-JH110088-PBV_SPIROMICS_EXPIRATION_PBV__0.5__QR40__RESEARCH__A_80KV_Qr40d_3_53062428"
Vx3Path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Vx3.nrrd"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/SPI-MU163450-PBV_FRC_0.625_MD STND_VNC.mask.nii.gz"
output_path = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'
filename = "Case-59-QC_Image.png"
final_filename = "Case-59-QC-Image(2CM).png"
Vx3_path = Vx3Path

# 3D Screenshot Stuff check for repeats
nrrd_file_path = labelpath
segmentation_path = labelpath
case_number = "Case-59-"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory
output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

# These filenames are just to have the program run all at the same time
Aseed_filename_base = case_number + "Aseed"
Vseed_filename_base = case_number + "Vseed"

vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path_seg + vtk_filename_aseed
output_filename_vseed = output_path_seg + vtk_filename_vseed
screenshot_filename = "Model.png"


models_folder = os.path.join(os.path.dirname(output_path), "QC-Images")
os.makedirs(models_folder, exist_ok=True)

# Load the image using nibabel to see image information
data, header = nrrd.read(Vx3_path)

# Print the metadata
print("Data Shape:", data.shape)
print("Data Type:", data.dtype)
print("Header:", header)

# Load the 3D image
Vx3 = sitk.ReadImage(Vx3_path)
print("Reading Vx3...")
# Get the size of the image in x, y, and z dimensions
size_x, size_y, size_z = Vx3.GetSize()
spacing = Vx3.GetSpacing()
y_index = Vx3.GetSize()[1] // 2
print("Spacing:", spacing)

# Extract the middle plane along the y-dimension
slice_2d = Vx3[:,y_index,:]
slice_array = sitk.GetArrayFromImage(slice_2d)

# Set the Window level
window_level_min = -1024  # Adjust these values according to your preference
window_level_max = 650

# Apply Window Level to image
windowed_image = sitk.IntensityWindowing(slice_2d, window_level_min, window_level_max, 0, 255)
windowed_slice_array = sitk.GetArrayFromImage(windowed_image)

#Show Vx3
plt.imshow(slice_array, cmap="gray",origin='lower')
plt.axis("off")
plt.show()
print("Window Level Adjusted!")

#Show Vx3 with correct window level
plt.imshow(windowed_slice_array, cmap="gray",origin='lower')
plt.axis("off")
plt.show()

# Save the Corrected Vx3 .PNG to output path
output_file_path = os.path.join(models_folder, filename)
plt.imsave(output_file_path, windowed_slice_array, cmap="gray", origin='lower')
print("Image saved at:", output_file_path)

# Load the image
image_path = output_file_path
output_path_png = models_folder + "/" + final_filename
image = Image.open(image_path)

# Determine line position
line_start_x = 65 # 65 pixels from the left
line_start_y = image.height - 35 # 35 pixels from the bottom

# Calculate line end position
line_end_x = line_start_x + int(20 / 0.38)  # 10 cm converted to pixels
line_end_y = line_start_y

# Draw the white line
draw = ImageDraw.Draw(image)
draw.line([(line_start_x, line_start_y), (line_end_x, line_end_y)], fill='white', width=2)
# # Save the modified image
image.save(output_path_png)

print("Scale bar added and image saved.")

#############################################################################################################################################################


def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)


# Load the .nrrd segmentation for Vseed
nrrd_reader_vseed = vtk.vtkNrrdReader()
nrrd_reader_vseed.SetFileName(segmentation_path)
nrrd_reader_vseed.Update()

vtk_image_data_vseed = vtk.vtkImageData()
vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

# Define scalar value for "Vseed" segment
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data_vseed)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)


print("Process Completed")

###################################################################################################################################################################

# 2 to fix (nearly done)
## THIS CODE WORKS TO MAKE A MODEL AND POSITION IT AT A FAVORABLE ANGLE TO TAKE AN IMAGE OF THE MAYAVI REPRESENTATION OF THE MODEL

lungmask_file = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/seg_net/Case-59-Lung-Mask-Model.vtk"
# Aseed_file = "z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/Case-59-Aseed.vtk"
Aseed_file = output_filename_aseed
segmentation_path = labelpath

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_filename = "z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/Case-59-Lung-Mask-Aseed-Model.png"

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(Aseed_file)
srclung = mlab.pipeline.open(lungmask_file)

surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfacelung = mlab.pipeline.surface(srclung, color=(230/255, 120/255, 41/255), opacity=.15)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
spacing = (0.38, 0.38, 0.38)

distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2


x, y, z = center[0], center[1], center[2]

ypt = y + 550
camera_focal_point = np.array([(x), (y), z])
camera_position = np.array([(x),(ypt),z])

print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_filename, magnification=magnification_factor)
print("Image Saved")

mlab.show()

 

In [ ]:
### WORKING DONT TOUCH. CREATE vseed if that equals 2

import vtk
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from PIL import Image, ImageDraw
from mayavi import mlab


def update_camera_info(obj, evt):
    camera = mlab.gcf().scene.camera
    print("Camera Position:", camera.position)
    print("Camera Focal Point:", camera.focal_point)
    print("Camera View Up:", camera.view_up)

labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
nrrd_file_path = labelpath
filename = "Case-28-QC_Image.png"
final_filename = "Case-28-QC-Image(2CM).png"
segmentation_path = labelpath
case_number = "Case-28-JH112953-SPI__0.75__QR40RESEARCH__A_80KV_Qr40d_53901001"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images/'  # Change this to your desired output directory


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed"
Vseed_model_base = "Vseed"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path_seg + vtk_filename_aseed
output_filename_vseed = output_path_seg + vtk_filename_vseed
screenshot_filename = "-Model.png"


# Load the .nrrd segmentation for Vseed
nrrd_reader_vseed = vtk.vtkNrrdReader()
nrrd_reader_vseed.SetFileName(segmentation_path)
nrrd_reader_vseed.Update()

vtk_image_data_vseed = vtk.vtkImageData()
vtk_image_data_vseed.DeepCopy(nrrd_reader_vseed.GetOutput())

# Define scalar value for "Vseed" segment
vseed_scalar_value = 2

# Create a Marching Cubes filter for "Vseed" segment
contour_vseed = vtk.vtkMarchingCubes()
contour_vseed.SetInputData(vtk_image_data_vseed)
contour_vseed.SetValue(0, vseed_scalar_value)
contour_vseed.Update()

# Write the "Vseed" mesh to a .vtk file
vtk_writer_vseed = vtk.vtkPolyDataWriter()
vtk_writer_vseed.SetFileName(output_filename_vseed) 
vtk_writer_vseed.SetInputData(contour_vseed.GetOutput())
vtk_writer_vseed.Write()

print("Vseed Segmentation conversion complete.")

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/Aseed.nrrd"

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation Conversion Complete")

#CHANGE THIS SELECT FOLDER TO SAVE TOO AND FILE NAME OF .PNG
screenshot_output = output_path_seg + case_number + screenshot_filename

mlab.figure(bgcolor=(1, 1, 1))
srcA = mlab.pipeline.open(output_filename_aseed)
srcV = mlab.pipeline.open(output_filename_vseed)

# Create a surface visualization of the data with blue color and 50% opacity
surfaceA = mlab.pipeline.surface(srcA, color=(0, 151/255, 206/255), opacity=1)
surfaceV = mlab.pipeline.surface(srcV, color=(216/255, 101/255, 79/255), opacity=1)

# Find Segmentation Center
model = sitk.ReadImage(segmentation_path)
origin = model.GetOrigin()
dimensions = model.GetSize()
direction = model.GetDirection()
spacing = (0.38, 0.38, 0.38)
distance = np.array(spacing) * np.array(dimensions)
center = np.array(origin) + np.array(distance) / 2

print("Origin: ", origin)
print("Dimensions: ", dimensions)
print("Center: ", center)

x, y, z = center[0], center[1], center[2]
ypt = y + 500

camera_focal_point = np.array([x, y, z])
camera_position = np.array([x,ypt,z])
print("Camera Position", camera_position)
print("Camera Focal Point", camera_focal_point)

# Set the camera parameters
# Write code to calculate this
mlab.gcf().scene.camera.position = [x, ypt, z]
mlab.gcf().scene.camera.focal_point = [x, y, z]
mlab.gcf().scene.camera.view_up = [0, 0, 1]

# Attach the callback to the interaction event
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

magnification_factor = 5
mlab.savefig(screenshot_output, magnification=magnification_factor)

# Comment this line below out if you don't want to open 3D viewer
mlab.show()

print("Process Completed")


In [ ]:

import vtk
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from PIL import Image, ImageDraw
from mayavi import mlab


labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-59-Spiromics-44556786/Markups/88-Nate/S-Final-1-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/seg_net/SPI-JH112953-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz"

Vx3_path = Vx3Path
# Output path for PNG of Vx3 
# Replace with the desired output file path
output_path = "z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/"
# filename = "Case-59-QC_Image.png"
# final_filename = "Case-59-QC-Image(2CM).png"

segmentation_path = labelpath
# case_number = "Case-59-JH112953-SPI__0.75__QR40RESEARCH__A_80KV_Qr40d_53901001"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(segmentation_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation conversion complete.")

In [ ]:

import vtk
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from PIL import Image, ImageDraw
from mayavi import mlab

Vx3Path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Vx3.nrrd"
labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Markup/88-Jensen/S-Final-2-label.nrrd"
maskpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/seg_net/SPI-JH112953-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz"

Vx3_path = Vx3Path
# Output path for PNG of Vx3 
# Replace with the desired output file path
output_path = "z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/"
filename = "Case-28-QC_Image.png"
final_filename = "Case-28-QC-Image(2CM).png"

segmentation_path = labelpath
case_number = "Case-28-JH112953-SPI__0.75__QR40RESEARCH__A_80KV_Qr40d_53901001"
output_path_seg = 'z:/Q- Figures&Reports/T32-Poster/Nate/PNG-Outputs/Segmentation Images'  # Change this to your desired output directory

# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)


# This line creates Models Forlder if not already made
if not os.path.exists(output_path_seg):
    os.makedirs(output_path_seg)

# These filenames are just to have the program run all at the same time
Aseed_filename_base = "Aseed"
Vseed_filename_base = "Vseed"
Aseed_model_base = "Aseed(decimated)"
Vseed_model_base = "Vseed(decimated)"
vtk_filename_aseed = f"{Aseed_filename_base}.vtk"
vtk_filename_vseed = f"{Vseed_filename_base}.vtk"
output_filename_aseed = output_path + vtk_filename_aseed
output_filename_vseed = output_path + vtk_filename_vseed


# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(segmentation_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 2

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()

print("Aseed Segmentation conversion complete.")

In [ ]:
import nrrd
import numpy as np

labelpath = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Markup/88-Jensen/S-Final-2-label.nrrd"

nrrd_file_path = labelpath

# Read the .nrrd file
data, header = nrrd.read(nrrd_file_path)

data[data == 2] = 0 

output_nrrd_file_path = "z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-28-0.75__QR40RESEARCH__A_80KV_Qr40d_53901001/Markup/88-Jensen/Aseed.nrrd"

# Write the modified data and header to a new .nrrd file
nrrd.write(output_nrrd_file_path, data, header)
print("Aseed.nrrd created!")
# Load the .nrrd segmentation for Aseed
nrrd_reader_aseed = vtk.vtkNrrdReader()
nrrd_reader_aseed.SetFileName(output_nrrd_file_path)
nrrd_reader_aseed.Update()

vtk_image_data_aseed = vtk.vtkImageData()
vtk_image_data_aseed.DeepCopy(nrrd_reader_aseed.GetOutput())

# Define scalar value for "Aseed" segment
aseed_scalar_value = 1

# Create a Marching Cubes filter for "Aseed" segment
contour_aseed = vtk.vtkMarchingCubes()
contour_aseed.SetInputData(vtk_image_data_aseed)
contour_aseed.SetValue(0, aseed_scalar_value)
contour_aseed.Update()

# Write the "Aseed" mesh to a .vtk file
vtk_writer_aseed = vtk.vtkPolyDataWriter()
vtk_writer_aseed.SetFileName(output_filename_aseed) 
vtk_writer_aseed.SetInputData(contour_aseed.GetOutput())
vtk_writer_aseed.Write()